In [1]:
import torch
import torch.nn as nn
from TorchCRF import CRF
import pandas as pd
import numpy as np
import pickle
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from collections import Counter
import ast
import os
from tqdm.auto import tqdm

TRAIN_BIO_PATH = "../../data/processed/train_bio.csv"
VAL_BIO_PATH = "../../data/processed/validation_bio.csv"
EMBEDDINGS_PATH = "../../data/external/embeddings/ru_en_aligned.pkl"
MODELS_DIR = "../../models/iteration-1/"

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Информация: Вычисления будут производиться на устройстве: {device}")

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f"Информация: Random seed ({SEED}) установлен для обеспечения воспроизводимости.")

print("\nДиагностика CUDA:")
print(f"- torch.cuda.is_available(): {torch.cuda.is_available()}")
print(f"- torch.version.cuda: {getattr(torch.version, 'cuda', None)}")
try:
    from torch.backends import cudnn
    print(f"- cudnn.enabled: {cudnn.enabled}, cudnn.version(): {cudnn.version() if hasattr(cudnn, 'version') else None}")
except Exception as e:
    print(f"- Информация по cuDNN недоступна: {e}")
print(f"- torch.cuda.device_count(): {torch.cuda.device_count()}")
for idx in range(torch.cuda.device_count()):
    try:
        print(f"  * [{idx}] {torch.cuda.get_device_name(idx)}")
    except Exception as e:
        print(f"  * [{idx}] ошибка чтения имени устройства: {e}")


Информация: Вычисления будут производиться на устройстве: cpu
Информация: Random seed (42) установлен для обеспечения воспроизводимости.

Диагностика CUDA:
- torch.cuda.is_available(): False
- torch.version.cuda: None
- cudnn.enabled: True, cudnn.version(): None
- torch.cuda.device_count(): 0


Гиперпараметры 

In [3]:
WORD_EMBEDDING_DIM = 300 
CHAR_EMBEDDING_DIM = 50  
CHAR_HIDDEN_DIM = 50      

LSTM_HIDDEN_DIM = 256
LSTM_NUM_LAYERS = 2 

BATCH_SIZE = 32
LEARNING_RATE = 1e-3
NUM_EPOCHS = 10 
DROPOUT_RATE = 0.5 

Создание модели

In [4]:
class CharEmbedding(nn.Module):
    def __init__(self, char_vocab_size, embedding_dim, hidden_dim, dropout_rate=0.25):
        super(CharEmbedding, self).__init__()
        self.embedding = nn.Embedding(char_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout_rate)
        print("Информация: Модуль CharEmbedding успешно инициализирован.")

    def forward(self, x):
        batch_size, seq_len, word_len = x.size()
        
        x = x.view(batch_size * seq_len, word_len)
        
        embedded = self.embedding(x)
        
        embedded = self.dropout(embedded)
        
        lstm_out, _ = self.lstm(embedded)
        
        output = lstm_out.permute(0, 2, 1)
        output = torch.max(output, 2)[0]
        
        output = output.view(batch_size, seq_len, -1)
        
        return self.dropout(output)

In [5]:
class BiLSTMCrfForNer(nn.Module):
    def __init__(self,
                 word_vocab_size,
                 word_embedding_dim,
                 char_vocab_size,
                 char_embedding_dim,
                 char_hidden_dim,
                 lstm_hidden_dim,
                 num_tags,
                 dropout_rate=0.33,
                 padding_idx=0):
        super(BiLSTMCrfForNer, self).__init__()

        self.word_embedding = nn.Embedding(
            num_embeddings=word_vocab_size,
            embedding_dim=word_embedding_dim,
            padding_idx=padding_idx
        )
        self.word_embedding.weight.requires_grad = False

        self.char_embedding = CharEmbedding(
            char_vocab_size=char_vocab_size,
            embedding_dim=char_embedding_dim,
            hidden_dim=char_hidden_dim,
            dropout_rate=dropout_rate
        )

        self.embedding_dropout = nn.Dropout(dropout_rate)

        self.lstm = nn.LSTM(
            input_size=word_embedding_dim + (2 * char_hidden_dim),
            hidden_size=lstm_hidden_dim,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=dropout_rate if 2 > 1 else 0
        )

        self.classifier = nn.Linear(2 * lstm_hidden_dim, num_tags)

        self.crf = CRF(num_tags=num_tags, batch_first=True)
        
        print("Информация: Основная модель BiLSTMCrfForNer успешно инициализирована.")

    def forward(self, word_ids, char_ids, mask, tags=None):
        # word_ids: (batch_size, seq_len)
        # char_ids: (batch_size, seq_len, word_len)
        # mask: (batch_size, seq_len)
        # tags: (batch_size, seq_len)

        word_embeds = self.word_embedding(word_ids)
        char_embeds = self.char_embedding(char_ids)
        
        combined_embeds = torch.cat([word_embeds, char_embeds], dim=-1)
        combined_embeds = self.embedding_dropout(combined_embeds)
        
        lstm_out, _ = self.lstm(combined_embeds)
        
        emissions = self.classifier(lstm_out)

        # Гарантируем булев тип маски для совместимости с PyTorch и CRF
        mask = mask.bool()

        if tags is not None:
            loss = -self.crf(emissions, tags, mask=mask, reduction='mean')
            return loss
        else:
            decoded_tags = self.crf.decode(emissions, mask=mask)
            return decoded_tags

Подача данных

In [6]:
class NerDataset(Dataset):
    def __init__(self, df_path, word2id=None, char2id=None, tag2id=None):
        self.df = pd.read_csv(df_path, sep=";")
        
        self.df['tokens'] = self.df['tokens'].apply(ast.literal_eval)
        self.df['tags'] = self.df['tags'].apply(ast.literal_eval)

        if word2id is None:
            self.word2id, self.id2word = self._build_vocab(self.df['tokens'])
        else:
            self.word2id, self.id2word = word2id, {v: k for k, v in word2id.items()}

        if char2id is None:
            all_chars = set("".join(["".join(tokens) for tokens in self.df['tokens']]))
            self.char2id, self.id2char = self._build_char_vocab(all_chars)
        else:
            self.char2id, self.id2char = char2id, {v: k for k, v in char2id.items()}

        if tag2id is None:
            # ВАЖНО: строим словарь ТЕГОВ без <UNK>, обязательно добавляем O
            self.tag2id, self.id2tag = self._build_tag_vocab(self.df['tags'])
        else:
            self.tag2id, self.id2tag = tag2id, {v: k for k, v in tag2id.items()}
            
        print(f"Информация: Dataset загружен. Размер: {len(self.df)} записей.")
        print(f"Информация: Размер словаря слов: {len(self.word2id)}")
        print(f"Информация: Размер словаря символов: {len(self.char2id)}")
        print(f"Информация: Размер словаря тегов: {len(self.tag2id)}")

    def _build_vocab(self, data):
        vocab = {"<PAD>": 0, "<UNK>": 1}
        for sequence in data:
            for item in sequence:
                if item not in vocab:
                    vocab[item] = len(vocab)
        id2vocab = {v: k for k, v in vocab.items()}
        return vocab, id2vocab

    def _build_char_vocab(self, chars):
        vocab = {"<PAD>": 0, "<UNK>": 1}
        for char in sorted(list(chars)):
            if char not in vocab:
                vocab[char] = len(vocab)
        id2vocab = {v: k for k, v in vocab.items()}
        return vocab, id2vocab

    # НОВОЕ: билдер словаря для тегов (без <UNK>), с обязательным 'O'
    def _build_tag_vocab(self, tags_series):
        tag_vocab = {"<PAD>": 0}
        # гарантируем наличие 'O'
        if "O" not in tag_vocab:
            tag_vocab["O"] = len(tag_vocab)
        for seq in tags_series:
            for tag in seq:
                if tag not in tag_vocab:
                    tag_vocab[tag] = len(tag_vocab)
        id2tag = {v: k for k, v in tag_vocab.items()}
        return tag_vocab, id2tag
    # ... existing code ...
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        tokens = row['tokens']
        tags = row['tags']

        word_ids = [self.word2id.get(token, self.word2id["<UNK>"]) for token in tokens]
        # ВАЖНО: неизвестные теги (на случай аномалий) отправляем в 'O', а не в <PAD>
        o_idx = self.tag2id.get("O", 0)
        tag_ids = [self.tag2id.get(tag, o_idx) for tag in tags]  # никаких паддингов в середине последовательности
        
        char_ids = []
        for token in tokens:
            ids = [self.char2id.get(char, self.char2id["<UNK>"]) for char in token]
            char_ids.append(ids)

        return {"words": word_ids, "chars": char_ids, "tags": tag_ids}

def collate_fn(batch, word_pad_idx=0, char_pad_idx=0, tag_pad_idx=0):
    max_seq_len = max(len(item['words']) for item in batch)
    max_word_len = max(max(len(char_seq) for char_seq in item['chars']) if item['chars'] else 0 for item in batch)

    padded_words, padded_chars, padded_tags, masks = [], [], [], []

    for item in batch:
        seq_len = len(item['words'])
        
        padded_words.append(item['words'] + [word_pad_idx] * (max_seq_len - seq_len))
        padded_tags.append(item['tags'] + [tag_pad_idx] * (max_seq_len - seq_len))
        
        masks.append([1] * seq_len + [0] * (max_seq_len - seq_len))
        
        padded_char_seq = []
        for char_seq in item['chars']:
            padded_char_seq.append(char_seq + [char_pad_idx] * (max_word_len - len(char_seq)))
        
        if seq_len < max_seq_len:
            for _ in range(max_seq_len - seq_len):
                padded_char_seq.append([char_pad_idx] * max_word_len)
        
        padded_chars.append(padded_char_seq)

    return {
        "words": torch.tensor(padded_words, dtype=torch.long),
        "chars": torch.tensor(padded_chars, dtype=torch.long),
        "tags": torch.tensor(padded_tags, dtype=torch.long),
        "mask": torch.tensor(masks, dtype=torch.bool)
    }

Цикл обучения и оценки

In [7]:
def calculate_class_weights(tags_series, tag2id):
    all_tags = [tag for seq in tags_series for tag in seq]
    tag_counts = Counter(all_tags)
    
    # Создаем веса. Используем сглаживание, чтобы избежать деления на ноль.
    # Более редкие классы получат больший вес.
    weights = torch.ones(len(tag2id), device=device)
    for tag, count in tag_counts.items():
        if tag in tag2id:
            # Вес обратно пропорционален частоте
            weights[tag2id[tag]] = 1.0 / (count + 1e-6) 
    
    # Нормализуем веса
    weights = weights / weights.sum()
    # Увеличим вес редких классов еще сильнее
    weights = weights.pow(0.5)

    print("Информация: Рассчитаны веса для классов:")
    for tag, i in tag2id.items():
        print(f"- {tag}: {weights[i]:.4f}")
        
    return weights

def train_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    # Прогресс-бар по батчам
    pbar = tqdm(dataloader, desc="Train", leave=False, dynamic_ncols=True)
    for batch in pbar:
        # Перенос данных на нужное устройство
        words = batch['words'].to(device)
        chars = batch['chars'].to(device)
        tags = batch['tags'].to(device)
        mask = batch['mask'].to(device)

        # Обнуление градиентов
        optimizer.zero_grad()
        
        # Прямой проход и вычисление потерь
        loss = model(words, chars, mask, tags)
        
        # Обратный проход
        loss.backward()
        
        # Шаг оптимизатора
        optimizer.step()
        
        total_loss += loss.item()
        # Обновляем подпись прогресс-бара
        pbar.set_postfix(loss=f"{loss.item():.4f}")
        
    return total_loss / len(dataloader)

# --- НОВОЕ: Entity-level оценка BIO ---

def _extract_entities_bio(tags_seq):
    """
    Превращает список BIO-тегов в множество сущностей вида (type, start, end),
    где start/end — индексы (включительно) в пределах последовательности.
    Нормализация BIO: одиночные 'I-X' без валидного 'B-X' считаем как начало новой сущности.
    Служебные теги ('O', '<PAD>', '<UNK>') игнорируются.
    """
    entities = set()
    cur_type = None
    start = None

    def close_entity(end_idx):
        nonlocal cur_type, start
        if cur_type is not None and start is not None:
            entities.add((cur_type, start, end_idx))
        cur_type, start = None, None

    for i, tag in enumerate(tags_seq):
        if tag in ("O", "<PAD>", "<UNK>") or not isinstance(tag, str):
            # Закрываем текущую сущность
            if cur_type is not None:
                close_entity(i - 1)
            continue

        if tag.startswith("B-"):
            # Закрываем предыдущую, открываем новую
            if cur_type is not None:
                close_entity(i - 1)
            cur_type = tag[2:]
            start = i
        elif tag.startswith("I-"):
            t = tag[2:]
            if cur_type == t and start is not None:
                # продолжаем ту же сущность
                pass
            else:
                # нарушение BIO: начинаем новую сущность
                if cur_type is not None:
                    close_entity(i - 1)
                cur_type = t
                start = i
        else:
            # Неверный/неизвестный формат — закрываем текущую
            if cur_type is not None:
                close_entity(i - 1)

    # Закрываем хвост, если открыт
    if cur_type is not None:
        close_entity(len(tags_seq) - 1)

    return entities

def _compute_entity_report(all_true_entities, all_pred_entities):
    """
    На вход — списки множеств сущностей по батчам/предложениям.
    Возвращает отчёт с precision/recall/f1 per type и micro/macro/weighted averages.
    """
    # Агрегируем по типам
    per_type = {}
    support_per_type = Counter()

    total_tp = total_fp = total_fn = 0

    for true_set, pred_set in zip(all_true_entities, all_pred_entities):
        # По типам
        types = set([t for (t, _, _) in true_set]) | set([t for (t, _, _) in pred_set])
        for t in types:
            true_t = {e for e in true_set if e[0] == t}
            pred_t = {e for e in pred_set if e[0] == t}
            tp = len(true_t & pred_t)
            fp = len(pred_t - true_t)
            fn = len(true_t - pred_t)

            if t not in per_type:
                per_type[t] = {"tp": 0, "fp": 0, "fn": 0}
            per_type[t]["tp"] += tp
            per_type[t]["fp"] += fp
            per_type[t]["fn"] += fn
            support_per_type[t] += len(true_t)

            total_tp += tp
            total_fp += fp
            total_fn += fn

    # Формируем метрики
    report = {}
    for t, c in per_type.items():
        tp, fp, fn = c["tp"], c["fp"], c["fn"]
        prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        rec = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0
        report[t] = {
            "precision": prec,
            "recall": rec,
            "f1-score": f1,
            "support": support_per_type[t],
        }

    # Micro
    micro_prec = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    micro_rec = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    micro_f1 = 2 * micro_prec * micro_rec / (micro_prec + micro_rec) if (micro_prec + micro_rec) > 0 else 0.0
    report["micro avg"] = {
        "precision": micro_prec,
        "recall": micro_rec,
        "f1-score": micro_f1,
        "support": sum(support_per_type.values()),
    }

    # Macro
    valid_types = [t for t in report.keys() if t not in ("micro avg", "macro avg", "weighted avg")]
    if valid_types:
        macro_prec = sum(report[t]["precision"] for t in valid_types) / len(valid_types)
        macro_rec = sum(report[t]["recall"] for t in valid_types) / len(valid_types)
        macro_f1 = sum(report[t]["f1-score"] for t in valid_types) / len(valid_types)
    else:
        macro_prec = macro_rec = macro_f1 = 0.0
    report["macro avg"] = {
        "precision": macro_prec,
        "recall": macro_rec,
        "f1-score": macro_f1,
        "support": sum(support_per_type.values()),
    }

    # Weighted
    total_support = sum(support_per_type.values())
    if total_support > 0:
        weighted_prec = sum(report[t]["precision"] * support_per_type[t] for t in valid_types) / total_support
        weighted_rec = sum(report[t]["recall"] * support_per_type[t] for t in valid_types) / total_support
        weighted_f1 = sum(report[t]["f1-score"] * support_per_type[t] for t in valid_types) / total_support
    else:
        weighted_prec = weighted_rec = weighted_f1 = 0.0
    report["weighted avg"] = {
        "precision": weighted_prec,
        "recall": weighted_rec,
        "f1-score": weighted_f1,
        "support": total_support,
    }

    return report

def eval_epoch_entities(model, dataloader, id2tag):
    """
    Entity-level оценка: извлекаем сущности из BIO-последовательностей
    и считаем метрики по сущностям.
    """
    model.eval()
    all_true_entities = []
    all_pred_entities = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Eval", leave=False, dynamic_ncols=True):
            words = batch['words'].to(device)
            chars = batch['chars'].to(device)
            tags = batch['tags'].to(device)
            mask = batch['mask'].to(device)
            
            # Получение предсказаний
            predictions = model(words, chars, mask)
            
            # По каждому предложению вырезаем по фактической длине (по mask)
            for i in range(len(predictions)):
                seq_len = mask[i].sum().item()
                true_ids = tags[i][:seq_len].cpu().tolist()
                pred_ids = predictions[i][:seq_len]

                true_tags = [id2tag[idx] for idx in true_ids]
                pred_tags = [id2tag[idx] for idx in pred_ids]

                true_ents = _extract_entities_bio(true_tags)
                pred_ents = _extract_entities_bio(pred_tags)

                all_true_entities.append(true_ents)
                all_pred_entities.append(pred_ents)

    # Готовим финальный отчёт
    return _compute_entity_report(all_true_entities, all_pred_entities)

Подготовка эмбеддингов

In [8]:
def load_and_prepare_embeddings(word2id, filepath, embedding_dim):
    with open(filepath, 'rb') as f:
        fasttext_model = pickle.load(f)

    embedding_matrix = np.random.uniform(-0.05, 0.05, (len(word2id), embedding_dim))
    
    hits = 0
    for word, i in word2id.items():
        if word in fasttext_model:
            embedding_matrix[i] = fasttext_model[word]
            hits += 1
    
    print(f"Информация: Найдено {hits} из {len(word2id)} слов в предобученной модели ({hits / len(word2id) * 100:.2f}%).")
    
    embedding_matrix[word2id["<PAD>"]] = np.zeros(embedding_dim)
    
    return torch.tensor(embedding_matrix, dtype=torch.float)

ОБУЧЕНИЕ

Загрузка данных

In [9]:
train_dataset = NerDataset(TRAIN_BIO_PATH)

val_dataset = NerDataset(VAL_BIO_PATH, 
                         word2id=train_dataset.word2id, 
                         char2id=train_dataset.char2id, 
                         tag2id=train_dataset.tag2id)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

print("\nИнформация: Датасеты и даталоадеры успешно созданы.")

embedding_weights = load_and_prepare_embeddings(train_dataset.word2id, EMBEDDINGS_PATH, WORD_EMBEDDING_DIM)

Информация: Dataset загружен. Размер: 23163 записей.
Информация: Размер словаря слов: 6108
Информация: Размер словаря символов: 101
Информация: Размер словаря тегов: 9
Информация: Dataset загружен. Размер: 4088 записей.
Информация: Размер словаря слов: 6108
Информация: Размер словаря символов: 101
Информация: Размер словаря тегов: 9

Информация: Датасеты и даталоадеры успешно созданы.
Информация: Найдено 3132 из 6108 слов в предобученной модели (51.28%).


Инициализация модели

In [10]:
if embedding_weights is not None:
    model = BiLSTMCrfForNer(
        word_vocab_size=len(train_dataset.word2id),
        word_embedding_dim=WORD_EMBEDDING_DIM,
        char_vocab_size=len(train_dataset.char2id),
        char_embedding_dim=CHAR_EMBEDDING_DIM,
        char_hidden_dim=CHAR_HIDDEN_DIM,
        lstm_hidden_dim=LSTM_HIDDEN_DIM,
        num_tags=len(train_dataset.tag2id),
        dropout_rate=DROPOUT_RATE,
        padding_idx=train_dataset.word2id["<PAD>"]
    )
    
    model.word_embedding.weight.data.copy_(embedding_weights)
    
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    print("\nИнформация: Модель и оптимизатор инициализированы.")

Информация: Модуль CharEmbedding успешно инициализирован.
Информация: Основная модель BiLSTMCrfForNer успешно инициализирована.

Информация: Модель и оптимизатор инициализированы.


Оценка обучения

In [ ]:
def eval_epoch(model, dataloader, id2tag):
    model.eval() 
    all_true_tags = []
    all_pred_tags = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Оценка на валидации"):
            words = batch['words'].to(device)
            chars = batch['chars'].to(device)
            tags = batch['tags'].to(device)
            mask = batch['mask'].to(device)
            
            predictions = model(words, chars, mask)
            
            for i in range(len(predictions)):
                seq_len = mask[i].sum().item()
                
                true_tags = tags[i][:seq_len].cpu().tolist()
                pred_tags = predictions[i][:seq_len]
                
                all_true_tags.extend([id2tag[tag_id] for tag_id in true_tags])
                all_pred_tags.extend([id2tag[tag_id] for tag_id in pred_tags])

    labels_to_include = [tag for tag in id2tag.values() if tag not in ["O", "<PAD>", "<UNK>"]]
    
    report = classification_report(
        all_true_tags, 
        all_pred_tags, 
        labels=labels_to_include,
        output_dict=True, 
        zero_division=0 
    )
    
    return report

Запуск обучения

In [11]:
best_val_f1 = 0.0
history = []

print("\n--- Начало процесса обучения ---")
# Прогресс-бар по эпохам
for epoch in tqdm(range(1, NUM_EPOCHS + 1), desc="Epochs", unit="epoch", dynamic_ncols=True):
    train_loss = train_epoch(model, train_dataloader, optimizer)
    
    # НОВОЕ: entity-level отчёт
    report = eval_epoch_entities(model, val_dataloader, train_dataset.id2tag)
    val_f1_macro = report['macro avg']['f1-score']
    val_precision_macro = report['macro avg']['precision']
    val_recall_macro = report['macro avg']['recall']
    
    history.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'val_f1': val_f1_macro,
        'val_precision': val_precision_macro,
        'val_recall': val_recall_macro
    })
    
    clear_output(wait=True)
    print(f"--- Эпоха {epoch}/{NUM_EPOCHS} --- (Длительность: {epoch_duration:.2f} сек)")
    print(f"  Потери на обучении (Train Loss): {train_loss:.4f}")
    print(f"  F1-macro (entity-level) на валидации: {val_f1_macro:.4f}")

    print("  Детальный отчёт по сущностям:")
    # Выводим только реальные типы (без агрегатов)
    for tag, metrics in report.items():
        if isinstance(metrics, dict) and tag not in {"micro avg", "macro avg", "weighted avg"}:
            print(f"    - {tag:<10}: F1={metrics['f1-score']:.4f}, Precision={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}, Support={metrics['support']}")

    print("  Агрегаты:")
    for agg in ("micro avg", "macro avg", "weighted avg"):
        m = report[agg]
        print(f"    - {agg:<12}: F1={m['f1-score']:.4f}, Precision={m['precision']:.4f}, Recall={m['recall']:.4f}, Support={m['support']}")

    if val_f1_macro > best_val_f1:
        best_val_f1 = val_f1_macro
        best_model_state = model.state_dict().copy()
        print(f"  Новый лучший результат! Модель сохранена (F1-macro entity-level: {best_val_f1:.4f}).")
        os.makedirs(MODELS_DIR, exist_ok=True)
        torch.save(best_model_state, os.path.join(MODELS_DIR, "bilstm_v1_best.pth"))

print("\n--- Обучение завершено ---")
print(f"Лучший F1-macro на валидации: {best_val_f1:.4f}")


--- Начало процесса обучения ---


Epochs:   0%|          | 0/10 [00:00<?, ?epoch/s]

Train:   0%|          | 0/724 [00:00<?, ?it/s]

Eval:   0%|          | 0/128 [00:00<?, ?it/s]


Эпоха 1/10:
  Потери на обучении (Train Loss): 1.7776
  F1-macro (entity-level) на валидации: 0.4001
  Детальный отчёт по сущностям:
    - TYPE      : F1=0.9239, Precision=0.8801, Recall=0.9723, Support=4372
    - BRAND     : F1=0.6765, Precision=0.8115, Recall=0.5801, Support=1143
    - VOLUME    : F1=0.0000, Precision=0.0000, Recall=0.0000, Support=17
    - PERCENT   : F1=0.0000, Precision=0.0000, Recall=0.0000, Support=1
  Агрегаты:
    - micro avg   : F1=0.8791, Precision=0.8702, Recall=0.8881, Support=5533
    - macro avg   : F1=0.4001, Precision=0.4229, Recall=0.3881, Support=5533
    - weighted avg: F1=0.8698, Precision=0.8631, Recall=0.8881, Support=5533
  Новый лучший результат! Модель сохранена (F1-macro entity-level: 0.4001).


...

Eval:   0%|          | 0/128 [00:00<?, ?it/s]


Эпоха 9/10:
  Потери на обучении (Train Loss): 0.4369
  F1-macro (entity-level) на валидации: 0.5385
  Детальный отчёт по сущностям:
    - TYPE      : F1=0.9477, Precision=0.9240, Recall=0.9728, Support=4372
    - BRAND     : F1=0.8252, Precision=0.8777, Recall=0.7787, Support=1143
    - VOLUME    : F1=0.3810, Precision=1.0000, Recall=0.2353, Support=17
    - PERCENT   : F1=0.0000, Precision=0.0000, Recall=0.0000, Support=1
  Агрегаты:
    - micro avg   : F1=0.9227, Precision=0.9153, Recall=0.9302, Support=5533
    - macro avg   : F1=0.5385, Precision=0.7004, Recall=0.4967, Support=5533
    - weighted avg: F1=0.9205, Precision=0.9145, Recall=0.9302, Support=5533


Train:   0%|          | 0/724 [00:00<?, ?it/s]

Eval:   0%|          | 0/128 [00:00<?, ?it/s]


Эпоха 10/10:
  Потери на обучении (Train Loss): 0.4117
  F1-macro (entity-level) на валидации: 0.5319
  Детальный отчёт по сущностям:
    - TYPE      : F1=0.9500, Precision=0.9309, Recall=0.9700, Support=4372
    - BRAND     : F1=0.8329, Precision=0.8418, Recall=0.8241, Support=1143
    - VOLUME    : F1=0.3448, Precision=0.4167, Recall=0.2941, Support=17
    - PERCENT   : F1=0.0000, Precision=0.0000, Recall=0.0000, Support=1
  Агрегаты:
    - micro avg   : F1=0.9240, Precision=0.9107, Recall=0.9376, Support=5533
    - macro avg   : F1=0.5319, Precision=0.5473, Recall=0.5221, Support=5533
    - weighted avg: F1=0.9238, Precision=0.9107, Recall=0.9376, Support=5533

--- Обучение завершено ---


Сохранение результатов

In [12]:
artefacts = {
    "word2id": train_dataset.word2id,
    "char2id": train_dataset.char2id,
    "tag2id": train_dataset.tag2id,
    "id2tag": train_dataset.id2tag
}

with open(ARTEFACTS_PATH, "wb") as f:
    pickle.dump(artefacts, f)

print(f"Информация: Словари для инференса (артефакты) успешно сохранены в: {ARTEFACTS_PATH}")


history_df = pd.DataFrame(history)

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 6)

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(history_df['epoch'], history_df['train_loss'], label='Потери на обучении', color='b', marker='o')
ax1.set_title('Динамика потерь на обучении по эпохам', fontsize=14)
ax1.set_xlabel('Эпоха', fontsize=12)
ax1.set_ylabel('Значение Loss', fontsize=12)
ax1.legend()
ax1.xaxis.set_major_locator(plt.MaxNLocator(integer=True)) 


ax2.plot(history_df['epoch'], history_df['val_f1'], label='F1-macro', color='g', marker='o')
ax2.plot(history_df['epoch'], history_df['val_precision'], label='Precision-macro', color='r', marker='s', linestyle='--')
ax2.plot(history_df['epoch'], history_df['val_recall'], label='Recall-macro', color='orange', marker='^', linestyle='--')
ax2.set_title('Динамика метрик на валидации по эпохам', fontsize=14)
ax2.set_xlabel('Эпоха', fontsize=12)
ax2.set_ylabel('Значение метрики', fontsize=12)
ax2.legend()
ax2.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

plt.savefig(CHART_PATH)

print(f"Информация: Графики процесса обучения сохранены в: {CHART_PATH}")

plt.show()


Информация: Лучшая модель и артефакты сохранены в директорию: ../../models/iteration-1/
